## Questions

- Why is the MCMC sampling so slow...?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.distributions.constraints as constraints

import pyro
import pyro.infer
import pyro.distributions as dist

from pyro.infer import MCMC, NUTS, Predictive

pyro.set_rng_seed(101)

%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [21]:
intercept_0 = 4
beta_0 = [2, 3] # the _0 represents the true parameter, not to be confused with the intercept
sigma_0 = 1.5
n = 20

In [31]:
x1 = np.exp(np.random.normal(loc=3, scale=2, size=[n, 1]))
x2 = np.random.binomial(n=1, p=0.4, size=[n, 1])
assert sum(x2) > 0 and sum(x2) < n

x = np.hstack((x1, x2))
y = intercept_0 + x@np.array(beta) + np.random.normal(loc=0, scale=sigma_0, size=[n,])

x = torch.Tensor(x)
y = torch.Tensor(y)

In [48]:
def regression(x, y):
    intercept = pyro.sample("intercept", dist.Normal(loc=0, scale=10))
    beta = []
    
    for i in range(x.shape[1]):
        beta.append(pyro.sample(f"beta{i}", dist.Normal(loc=0, scale=10)))
        
    sigma = pyro.sample("sigma", dist.InverseGamma(concentration=4, rate=2))
    
    mean = intercept + x.matmul(torch.Tensor(beta))
    
    return pyro.sample("y", dist.Normal(loc=mean, scale=sigma), obs=y)

In [49]:
nuts_kernel = NUTS(model=regression)
mcmc = MCMC(kernel=nuts_kernel,
                 num_samples=10000,
                 num_chains=1, 
                 warmup_steps=1000)
posterior = mcmc.run(x=x, y=y)

Warmup:   3%|▎         | 299/11000 [13:39,  3.08s/it, step size=8.27e-04, acc. prob=0.740]

KeyboardInterrupt: 